In [6]:
import numpy as np
from numba import njit
from numba.experimental import jitclass
from numba import int32, float32

In [7]:
spec = [
    ('L', int32),
    ('mag', int32[:]),
    ('T', float32),
    ('j', int32)
]
rng = np.random.default_rng(seed=None)
@jitclass(spec)
class Ising:
    # mag: np.ndarray
    # L: int
    # T: float
    # j: int
    # @njit
    def __init__(self, L, T) -> None:
        # self.rng = np.random.default_rng(seed=None)
        self.mag = self.CreateMagnet(L)
        self.L = L
        self.T = T
        self.j = 1

    # @njit
    def CreateMagnet(self, L):
        mag = rng.choice([1, -1], (L, L), dtype=int32)
        
        return mag
    
    # @njit
    def takeStep(self):
        # can also loop through in order
        x = rng.integers(0, self.L)
        y = rng.integers(0, self.L)
        deltaE = self.checkEnergyChange(x, y)
        r = rng.random()
        if (r < np.exp(-deltaE/self.T)):
            self.mag[x, y] *= -1
    
    # @njit
    def checkEnergyChange(self, x, y):
        Ebefore = 0
        # if they are both 1 or -1, then their product will be 1, but we want E to be negative
        Ebefore += self.mag[x, y] * self.mag[x, (y+1) % self.L] * -self.j
        Ebefore += self.mag[x, y] * self.mag[x, (y-1) % self.L] * -self.j
        Ebefore += self.mag[x, y] * self.mag[(x+1) % self.L, y] * -self.j
        Ebefore += self.mag[x, y] * self.mag[(x-1) % self.L, y] * -self.j
        
        Eafter = 0
        Eafter += self.mag[x, y] * self.mag[x, (y+1) % self.L] * self.j
        Eafter += self.mag[x, y] * self.mag[x, (y-1) % self.L] * self.j
        Eafter += self.mag[x, y] * self.mag[(x+1) % self.L, y] * self.j
        Eafter += self.mag[x, y] * self.mag[(x-1) % self.L, y] * self.j
        
        return Eafter - Ebefore
    
    # @njit
    def sweep(self):
        for i in range(self.L * self.L):
            self.takeStep()

    # @njit
    def totalEnergy(self):
        E = 0
        for i in range(self.L):
            for k in range(self.L):
                E += self.mag[i, k] * self.mag[i, (k+1) % self.L] * -self.j
                E += self.mag[i, k] * self.mag[(i+1) % self.L, k] * -self.j
                
        return E

In [43]:
rng = np.random.default_rng(seed=None)
def CreateMagnet(L):
    mag = rng.choice([1, -1], (L, L))
    return mag

@njit
def takeStep(mag, T):
    # can also loop through in order
    L, L2 = mag.shape
    x = np.random.randint(0, L)
    y = np.random.randint(0, L)
    # x = rng.integers(0, L)
    # y = rng.integers(0, L)
    deltaE = checkEnergyChange(mag, x, y)
    # r = rng.random()
    r = np.random.randint(0, 10000) / 10000
    if (r < np.exp(-deltaE/T)):
        mag[x, y] *= -1
@njit
def checkEnergyChange(mag, x, y):
    j = 1
    L, L2 = mag.shape
    Ebefore = 0
    # if they are both 1 or -1, then their product will be 1, but we want E to be negative
    Ebefore += mag[x, y] * mag[x, (y+1) % L] * -j
    Ebefore += mag[x, y] * mag[x, (y-1) % L] * -j
    Ebefore += mag[x, y] * mag[(x+1) % L, y] * -j
    Ebefore += mag[x, y] * mag[(x-1) % L, y] * -j
    
    Eafter = 0
    Eafter += mag[x, y] * mag[x, (y+1) % L] * j
    Eafter += mag[x, y] * mag[x, (y-1) % L] * j
    Eafter += mag[x, y] * mag[(x+1) % L, y] * j
    Eafter += mag[x, y] * mag[(x-1) % L, y] * j
    
    return Eafter - Ebefore
@njit
def sweep(mag, T):
    L, L2 = mag.shape
    for i in range(L * L):
        takeStep(mag, T)
@njit
def totalEnergy(mag):
    L, L2 = mag.shape()
    E = 0
    j = 1
    for i in range(L):
        for k in range(L):
            E += mag[i, k] * mag[i, (k+1) % L] * -j
            E += mag[i, k] * mag[(i+1) % L, k] * -j
            
    return E

def Renormalize(mag, window):
    L = mag.shape[0]
    if (L % window != 0):
        raise ValueError()
    targetL = int(L / window)
    out = np.zeros((targetL, targetL))
    for i in range(0, targetL):
        for j in range(0, targetL):
            small = mag[i * window: i * window + window, j * window: j * window + window]
            print(sum(sum(small)))
    small = mag[0:window, 0:window]
    return small

In [44]:
myMag = CreateMagnet(9)
print(myMag)
T = 100
for i in range(100):
    sweep(myMag, T)
print(myMag)

[[ 1  1 -1 -1 -1 -1 -1 -1 -1]
 [-1 -1 -1 -1  1 -1 -1 -1  1]
 [ 1 -1  1  1 -1 -1  1  1 -1]
 [-1 -1 -1  1  1  1 -1  1  1]
 [ 1  1  1 -1 -1  1  1  1  1]
 [-1  1 -1  1  1  1  1 -1  1]
 [-1 -1  1 -1  1 -1 -1  1 -1]
 [-1 -1  1 -1  1 -1  1  1 -1]
 [-1 -1  1  1 -1 -1  1  1  1]]
[[-1 -1 -1  1 -1  1 -1  1  1]
 [ 1 -1  1  1 -1 -1 -1 -1  1]
 [-1  1 -1  1  1  1  1 -1 -1]
 [-1  1 -1 -1 -1  1 -1  1 -1]
 [-1  1 -1  1 -1 -1  1  1  1]
 [ 1 -1  1  1  1 -1  1  1  1]
 [-1 -1  1  1  1 -1 -1  1 -1]
 [-1  1 -1 -1 -1  1  1  1  1]
 [ 1 -1 -1  1 -1  1 -1 -1  1]]


In [45]:
print(Renormalize(myMag, 3))

-3
3
-1
-1
-1
5
-3
1
1
[[-1 -1 -1]
 [ 1 -1  1]
 [-1  1 -1]]
